In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
employ_file= "DP_LIVE_06082019033543494.csv"

In [6]:
employ_df = pd.read_csv(employ_file)
employ_df.head()


,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,EMP,MEN,THND_PER,A,1965,3346.5,NaN
1,AUS,EMP,MEN,THND_PER,A,1966,3362.5,NaN
2,AUS,EMP,MEN,THND_PER,A,1967,3412.5,NaN
3,AUS,EMP,MEN,THND_PER,A,1968,3474.0,NaN
4,AUS,EMP,MEN,THND_PER,A,1969,3548.0,NaN


In [8]:
employ_new_df=employ_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
employ_new_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
0,AUS,EMP,MEN,1965,3346.5
1,AUS,EMP,MEN,1966,3362.5
2,AUS,EMP,MEN,1967,3412.5
3,AUS,EMP,MEN,1968,3474.0
4,AUS,EMP,MEN,1969,3548.0


In [10]:
employ_group=employ_new_df.groupby(["LOCATION"])
employ_group